In [25]:
import demoji
from langdetect import detect
from datetime import datetime, timedelta


def remove_emojis(text):
    try:
        return demoji.replace(text.replace("\n", "; "), '')
    except:
        return None
                  
def detect_language(x):
    try:
        return detect(x)
    except:
        return None
    
def update_datetime(x):
    if "ago" in x :
        x = list(x)
        if x[0] == 'a':
            x[0] = '1'
        x = ''.join(x)   
        
        if "minute" in x:
            delta = timedelta(days=0)
        elif "hour" in x:
            delta = timedelta(days=0)
        elif "day" in x:
            delta = timedelta(days=int(x[0]))
        if "week" in x:
            delta = timedelta(weeks=int(x[0]))
        elif "month" in x:
            delta = timedelta(weeks=int(x[0])*4)
        elif "year" in x:
            delta = timedelta(weeks=int(x[0])*4*12)
        current_time = datetime.now()
    
        # Calculate the previous time
        result_time = current_time - delta  
        # adjust format
        date_str = result_time.strftime("%Y-%m-%d %H:%M")  
        return date_str
    else:
        datetime_obj = datetime.strptime(x, " %B %Y")
        # adjust format
        formatted_datetime = datetime_obj.strftime("%Y-%m-%d %H:%M")
        return formatted_datetime
    
from transformers import pipeline
from googletrans import Translator
# Init Translator Object
translator = Translator()
sentiment_pipeline = pipeline("sentiment-analysis")    
def sentiment_analysis_review(review,language_src):
    
    try :
        if review == None or language_src == None or review.isspace() or review == "" :
            return (None, None)
        #translate review text to destination language
        translated_text = translator.translate(review, src=language_src, dest='en').text
        resutl_sentiment = sentiment_pipeline(translated_text)[0] #[{label : 'POSITIVE', score : value}, ]
        return resutl_sentiment.values()
    except Exception as e:
        print(f'There is an Exception : {e}')
        return (None, None)



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


## Get data from Tripadvisor

In [20]:
results = [] # <--- global list for items


import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.common.exceptions import StaleElementReferenceException


int_url     = "https://www.tripadvisor.com/ShowUserReviews-g293925-d12130500-r923315228-Poke_Saigon-Ho_Chi_Minh_City.html"
driver = webdriver.Chrome()
driver.get(int_url)

def get_reviews_per_page(driver):
    
    reviews=driver.find_elements(By.CLASS_NAME, "review-container")
    result = []
    for i in range(len(reviews)): 
        wait = WebDriverWait(driver, 20)   
        # Wait for the reviews to load completely
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.username.mo")))
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.prw_rup.prw_reviews_text_summary_hsx")))
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.rating.reviewItemInline > span")))
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.wrap > div.prw_rup.prw_reviews_stay_date_hsx")))

        user = reviews[i].find_element(By.CSS_SELECTOR,"div.username.mo").text
        text_review = reviews[i].find_element(By.CSS_SELECTOR,"div.prw_rup.prw_reviews_text_summary_hsx").text
        rate_element = reviews[i].find_element(By.CSS_SELECTOR,"div.rating.reviewItemInline > span")
        rate = rate_element.get_attribute("class").split("_")[-1][0]
        time_visit = reviews[i].find_element(By.CSS_SELECTOR,"div.wrap > div.prw_rup.prw_reviews_stay_date_hsx").text.split(":")[-1]  
   
        #Preprocessing 
        text_review = remove_emojis(text_review)
        language =  detect_language(text_review)
        sentiment, score = sentiment_analysis_review(text_review,language)
        time_visit = update_datetime(time_visit)
        item = {
            'user' : user,
            'text_review': text_review,
            'sentiment_review':sentiment,
            'sentiment_score' :score,
            'rate' : rate,
            'time_visit' : time_visit,
            'language': language,
            'platform': 'tripadvisor'
        }

        result.append(item) # <--- add to global list
        
    return result    


i=1 
driver.implicitly_wait(25)
while True:
    try:
        results += get_reviews_per_page(driver)
    except (TimeoutException, WebDriverException) as e:
        results += get_reviews_per_page(driver)
        print(next_page_url)
        break
    
    next_page_url = driver.find_element(By.CSS_SELECTOR,"a.nav.next").get_attribute("href")
    if next_page_url:
        driver.refresh()
        driver.get(next_page_url)
        driver.implicitly_wait(10)
        i+=1
        print("Navigating to Next Page ==> ", i )
    else:
        break
    time.sleep(3)
    
print("total pages :", i )
driver.quit()



Navigating to Next Page ==>  2
Navigating to Next Page ==>  3
Navigating to Next Page ==>  4
Navigating to Next Page ==>  5
Navigating to Next Page ==>  6
Navigating to Next Page ==>  7
Navigating to Next Page ==>  8
Navigating to Next Page ==>  9
Navigating to Next Page ==>  10
Navigating to Next Page ==>  11


Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


Có một exception xảy ra: The size of tensor a (570) must match the size of tensor b (512) at non-singleton dimension 1
en I obviously live on another planet looking at the other reviews. I went today for lunch given all the hype but for me it was good but not amazing. Their located in The Cafe Apartments on the main Walking Street in Saigon (second floor). Small branch, limited seating. Think wooden seats, beach vibe Hawaii. I stupidly went at peak period lunchtime 13:00. Suggest going outside of main mealtimes. I luckily got a seat. You can eat in or take out. The line was long with two members of staff who were trying their best. More a place to eat and go rather than linger for ages. There were only two suggested 'ready-made' options. None appealed to me. I think if they have more ready-made choices, lines would be less. Most people like me made their own bowls which obviously took longer. Those that went for the suggested ready-made standard options were a lot quicker. Choose your 

In [21]:
import pandas as pd        
df = pd.DataFrame(results).sort_values(by='time_visit') # <--- convert list to DataFrame
df.to_csv("Data/tripadvisor.csv",index=False, encoding="utf-8")

### Get data from Google Map 


In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

# Specify the URL of the business page on Google Maps
url = "https://www.google.com/maps/place/Poke+S%C3%A0i+G%C3%B2n+L%C3%BD+T%E1%BB%B1+Tr%E1%BB%8Dng/@10.7771788,106.6893238,14.47z/data=!4m8!3m7!1s0x31752f47f450b265:0xa46f765f0dea01e8!8m2!3d10.7771626!4d106.7001507!9m1!1b1!16s%2Fg%2F11g020cwyh?entry=ttu"
# Create an instance of the Chrome driver
driver = webdriver.Chrome()

# Navigate to the specified URL
driver.get(url)

# Wait for the reviews to load
wait = WebDriverWait(driver, 20)  # Increased the waiting time  

# Scroll down to load more reviews
body = driver.find_element(By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")
num_reviews = len(driver.find_elements(By.CLASS_NAME, 'jftiEf'))
while True:
    body.send_keys(Keys.END)
    time.sleep(5)  # Adjust the delay based on your internet speed and page loading time
    new_num_reviews = len(driver.find_elements(By.CLASS_NAME, 'jftiEf'))
    if new_num_reviews == num_reviews:
        # Scroll to the top to ensure all reviews are loaded
        body.send_keys(Keys.HOME)
        time.sleep(10)
        break
    num_reviews = new_num_reviews

# Wait for the reviews to load completely
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'jftiEf')))
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wiI7pd')))
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'kvMYJc')))
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'rsqaWe')))


[<selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2381")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2382")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2383")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2384")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2385")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2b368e0ccc0e7c5a8", element="D8A670C01FA282F87AFE854D6C014643_element_2386")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9d506397e11675a2

In [26]:
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
# Extract the text of each review
review_elements = driver.find_elements(By.CLASS_NAME, 'jftiEf')
result=[]
for review in review_elements:
    
        user = review.get_attribute("aria-label")
        #text_review = review.find_element(By.CSS_SELECTOR,"span.wiI7pd").text
        try:
                text_review = review.find_element(By.CSS_SELECTOR,"span.wiI7pd").text
        except (TimeoutException, WebDriverException,NoSuchElementException) as e:
                text_review = None
        rate_element = review.find_element(By.CSS_SELECTOR,"span.kvMYJc").get_attribute("aria-label")
        rate = rate_element.split(" ")[0]
        time_visit = review.find_element(By.CSS_SELECTOR,"span.rsqaWe").text

        #preprocessing 
        text_review = remove_emojis(text_review)
        language =  detect_language(text_review)
        sentiment, score =sentiment_analysis_review(text_review, language)
        time_visit = update_datetime(time_visit)

       
        item = {
            'user' : user,
            'text_review': text_review,
            'sentiment_review':sentiment,
            'sentiment_score' :score,
            'rate' : rate,
            'time_visit' : time_visit,
            'language': language,
            'platform': 'google_map'
        }

        result.append(item)
        time.sleep(2)


review None !!!
review None !!!
review None !!!
review None !!!


In [27]:
import pandas as pd        
df = pd.DataFrame(result).sort_values(by='time_visit') # <--- convert list to DataFrame
df.to_csv("Data/google_map.csv",index=False, encoding="utf-8")

## Combine the 2 tables together


In [13]:
import pandas as pd
import time
web_data = pd.read_csv("Data/tripadvisor.csv",index_col=None)
web_dataGGMap = pd.read_csv("Data/google_map.csv",index_col=None)


data_combine = pd.concat([web_data,web_dataGGMap], ignore_index=True).sort_values(by='time_visit')
data_combine.to_csv("Data/feedback_final.csv",index=False, encoding="utf-8")